## Para baixar os dados

Primeiro, instale a versão mais recente da [API do kaggle](https://www.kaggle.com/docs/api), configure corretamente e execute o comando abaixo:

```sh
kaggle datasets download kdd2020-cpr
```

Alternativamente, o download pode ser feito diretamente a partir [dessa página](https://www.kaggle.com/maffei2443/kdd2020-cpr).

Depois, basta extrair os dados para a pasta `kdd2020-cpr`.

**NOTA:*** O *DATASET* utilizado neste notebook contém os mesmos dados do [*dataset* da competição][dataset], sendo a única diferença que os dados estão já agrupados por ano (um csv por ano) e há como ler os dados utilizando-se dos tipos de dados que gastam o mínimo de memória possível.

[dataset]: https://www.kaggle.com/c/kddbr-2020/data

In [ ]:
import pickle
import itertools    # Para as variáveis categóricas
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
import datetime

Abaixo, alguns parâmetros "globais" que usamos durante a competição. Por exemplo, a constante que usamos para substituir **NaN** que apareciam nos dados (necessário já que o **MultiOutputRegressor** não aceitava dados com **NaN**.

In [ ]:
# Global params
INPUT_NA = -1
TNAME = str(datetime.datetime.now())[:-7]
YEARS = [2018]
OUTPUT_FILLER = 0
QTD = None

In [ ]:
def load_year(y):
    """Loads data relative to `year` year.

    Index columns is assumed to be `id` and `date` to be parseable
    by pandas engine into a `timestamp`list.
    """
    dtypes = pickle.load(open(
        f'../input/kdd2020-cpr/{y}_dtypes.pkl', 'rb'
    ))
    del dtypes['date']
    df = pd.read_csv(
        f'../input/kdd2020-cpr/{y}.csv',
        dtype=dtypes, parse_dates=['date'], index_col='id'
    )
    return df

Algumas features foram tomadas deste [ótimo notebook][notebook]; as demais, criadas pelo time.

[notebook]: https://www.kaggle.com/adrianoavelar/kdd-starter-kernel

In [ ]:
def create_features(df):
    """Creates some temporal features. Assumes a `date` columns with `pd.datetime` type
    or convertible to it."""
    def is_weekend(num):
        return num > 5
    df['date'] = pd.to_datetime( df.date )
    dt = df['date'].dt
    df['input_week'] = dt.week
    df['input_weekday'] = dt.weekday + 1
    df['input_weekday_sin'] = np.sin(2*np.pi*df.date.dt.weekday/7)    
    df['input_weekofyear'] = dt.weekofyear
    df['input_weekofyear_sin'] = np.sin(2*np.pi*dt.weekofyear/52)
    df['input_weekend'] = dt.weekday.apply(is_weekend)
    df['input_month'] = df.date.dt.month
    df['input_year'] = df.date.dt.year
    df['input_day'] = df.date.dt.day
    df['input_dt_sin_quarter']     = np.sin(2*np.pi*df.date.dt.quarter/4)
    df['input_dt_sin_day_of_week'] = np.sin(2*np.pi*df.date.dt.dayofweek/6)
    df['input_dt_sin_day_of_year'] = np.sin(2*np.pi*df.date.dt.dayofyear/365)
    df['input_dt_sin_day']         = np.sin(2*np.pi*df.date.dt.day/30)
    df['input_dt_sin_month']       = np.sin(2*np.pi*df.date.dt.month/12)
    return df

### Carregamos os conjuntos de teste e treinamento...

In [ ]:
base = pd.concat([load_year(year) for year in YEARS])
df_test = load_year(2019)

Faz-se a engenharia de atributos...

In [ ]:
create_features(df_test)
create_features(base)

### Separamos as colunas que devem ser usadas para predição e para serem preditas...

In [ ]:
input_columns = base.columns[base.columns.str.contains('input') ]
output_columns = base.columns[base.columns.str.contains('output') ]

### Conjunto de treinamento.

In [ ]:
X = base[input_columns].fillna(INPUT_NA).values
Y = base[output_columns].fillna(OUTPUT_FILLER).values

Os parêmtros que cada regressos XGB possuirá. Note-se a presença de todos os atributos que esse regressor pode receber. Entretando, a busca por melhores híperparâmetros deu-se apenas nos primeiros parâmetros listados, sendo os outros preenchidos com seu valor padrão na versão da biblioteca utilizada na competição.

In [ ]:
MODEL_PARAMS = {
 'colsample_bylevel': .5,
 'colsample_bynode': 0.5,
 'colsample_bytree': 0.5,
 'learning_rate': 0.01,
 'max_depth': 12,
    
 'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'gamma': 0,
 'gpu_id': 0,
 'importance_type': 'gain',
 'interaction_constraints': '',
 
 'max_delta_step': 0,
 
 'min_child_weight': 1,
 'missing': 6358,
 'monotone_constraints': '()',
 'n_estimators': 1000,
 'n_jobs': -1,
 'num_parallel_tree': 1,
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 0.2,
 'tree_method': 'gpu_hist',
 'validate_parameters': 1,
 'verbosity': 1,
 'sampling_method': 'gradient_based'
}

### Uma vez que o XGBoost não produz múltiplas saídas, utilizamos um *wrapper* disponibilizado pelo **sklearn** por este ser de fácil utilização. Basicamente, ele treina uma instância do modelo recebido pelo consrutor para cada variável de saída.

In [ ]:
clf = MultiOutputRegressor( XGBRegressor(**MODEL_PARAMS) )
clf.fit(X[:QTD], Y[:QTD])

In [ ]:
input_columns = df_test.columns[
    df_test.columns.str.contains('input')
]
df_test = df_test[input_columns].copy()
df_test.fillna(INPUT_NA, inplace=True)

In [ ]:
pred = clf.predict(df_test.values)

Para efetiva submissão, era esperado que as todas as predições estivessem dispostas em duas colunas: **id** e **value**. Contudo, a primeira na verdade contém, em cada linha, três informações: um *id* do *conjunto de teste*, seguido pelo caractere `_`, concatenado com um inteiro, também seguido por `_`, concatenado com outro inteiro.  
Esses dois inteiros representam respectivamente o número da *feature* predita (112 para cada dia) e a quantos dias no futuro se refere essa predição.  
  
A célula abaixo cria um *DataFrame* segundo essa lógica.

In [ ]:
pred_sub = pd.DataFrame(pred)
pred_sub.columns = output_columns
pred_sub['id'] = df_test.index

submission = []
for i, row in pred_sub.iterrows():
    for column, value in zip(output_columns, row.values):
        _id = "{}_{}".format(int(row.id), column)
        submission.append([_id, value])


df_sub = pd.DataFrame(submission)
df_sub.columns = ['id', 'value']
df_sub.set_index('id', inplace=True)

In [ ]:
submission.to_csv('{}.csv'.format(TNAME), index='id')

# Para submissão na última célula
submission.to_csv('submission.csv'.format(TNAME), index='id')

A céulua abaixo não tem utilidade aparente; porém, ao longo de nossos experimentos realizados no *Euler*, foi uma forma para documentarmos quais híperparâmetros produziram quais resultados. Lembrando que não foi feita uma otimização sistemática deles.

In [ ]:
print("MODEL PARAMS:", MODEL_PARAMS)
print("{}.csv".format(TNAME))

In [ ]:
!kaggle competitions submit -c kddbr-2020 -f submission.csv -m "Message"